# GBI

## 目标
通过 GBI sdk 接口完成选表和问表的能力。 

## 准备工作
### 平台注册
1.先在appbuilder平台注册，获取token
2.安装appbuilder-sdk

In [1]:
# !pip install appbuilder-sdk

In [18]:
import logging
import os

#  设置环境变量
os.environ["APPBUILDER_TOKEN"] = "***"


## 开发过程

### 设置表的 schema

In [3]:
SUPER_MARKET_SCHEMA = """
```
CREATE TABLE `supper_market_info` (
  `订单编号` varchar(32) DEFAULT NULL,
  `订单日期` date DEFAULT NULL,
  `邮寄方式` varchar(32) DEFAULT NULL,
  `地区` varchar(32) DEFAULT NULL,
  `省份` varchar(32) DEFAULT NULL,
  `客户类型` varchar(32) DEFAULT NULL,
  `客户名称` varchar(32) DEFAULT NULL,
  `商品类别` varchar(32) DEFAULT NULL,
  `制造商` varchar(32) DEFAULT NULL,
  `商品名称` varchar(32) DEFAULT NULL,
  `数量` int(11) DEFAULT NULL,
  `销售额` int(11) DEFAULT NULL,
  `利润` int(11) DEFAULT NULL
) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4
```
"""

PRODUCT_SALES_INFO = """
现有 mysql 表 product_sales_info, 
该表的用途是: 产品收入表
```
CREATE TABLE `product_sales_info` (
  `年` int,
  `月` int,
  `产品名称` varchar,
  `收入` decimal,
  `非交付成本` decimal,
  `含交付毛利` decimal
)
```
"""

# schema 和表名的映射
SCHEMA_MAPPING = {
    "supper_market_info": SUPER_MARKET_SCHEMA,
    "PRODUCT_SALES_INFO": PRODUCT_SALES_INFO
}

设置表的描述用于选表

In [4]:
table_descriptions = {
    "supper_market_info": "超市营收明细表，包含超市各种信息等",
    "product_sales_info": "产品销售表"
}

### 选表

In [19]:
import appbuilder
from appbuilder.core.message import Message
from appbuilder.core.components.gbi.basic import SessionRecord

# 生成问表对象
select_table = appbuilder.SelectTable(model_name="ERNIE-Bot 4.0", table_descriptions=table_descriptions)
query = "列出超市中的所有数据"
msg = Message({"query": query})
select_table_result_message = select_table(msg)
print(f"选的表是: {select_table_result_message.content}")

选的表是: ['supper_market_info']


### 问表
基于上面选出的表，通过获取表的 schema 进行问表

In [6]:
table_schemas = [SCHEMA_MAPPING[table_name] for table_name in select_table_result_message.content]
gbi_nl2sql = appbuilder.NL2Sql(model_name="ERNIE-Bot 4.0", table_schemas=table_schemas)
nl2sql_result_message = gbi_nl2sql(Message({"query": "列出超市中的所有数据"}))
print(f"sql: {nl2sql_result_message.content.sql}")
print("-----------------")
print(f"llm result: {nl2sql_result_message.content.llm_result}")

sql: SELECT * FROM supper_market_info;
-----------------
llm result: ```sql
SELECT * FROM supper_market_info;
```


设置 session

In [7]:
session = list()
session.append(SessionRecord(query=query, answer=nl2sql_result_message.content))

再次问表

In [8]:
nl2sql_result_message2 = gbi_nl2sql(Message({"query": "查看商品类别是水果的所有数据", 
                                             "session": session}))
print(f"sql: {nl2sql_result_message2.content.sql}")
print("-----------------")
print(f"llm result: {nl2sql_result_message2.content.llm_result}")

sql: SELECT * FROM supper_market_info WHERE 商品类别 = '水果'
-----------------
llm result: ```sql
SELECT * FROM supper_market_info WHERE 商品类别 = '水果'
```


### 增加列选优化
实际上数据中 "商品类别" 存储的是 "新鲜水果", 那么就可以通过列选的限制来优化 sql.

In [9]:
from appbuilder.core.components.gbi.basic import ColumnItem


column_constraint = [ColumnItem(ori_value="水果", 
                               column_name="商品类别", 
                               column_value="新鲜水果", 
                               table_name="supper_market_info", 
                               is_like=False)]

nl2sql_result_message2 = gbi_nl2sql(Message({"query": "查看商品类别是水果的所有数据",
                                    "column_constraint": column_constraint}))

print(f"sql: {nl2sql_result_message2.content.sql}")
print("-----------------")
print(f"llm result: {nl2sql_result_message2.content.llm_result}")

sql: SELECT * FROM supper_market_info WHERE 商品类别 = '新鲜水果'
-----------------
llm result: ```sql
SELECT * FROM supper_market_info WHERE 商品类别 = '新鲜水果'
```


从上面我们看到，商品类别不在是 "水果" 而是 修订为 "新鲜水果"

### 增加知识优化
当计算某些特殊知识的时候，大模型是不知道的，所以需要告诉大模型具体的知识，比如:
利润率的计算方式: 利润/销售额
可以将该知识注入。具体示例如下:

In [10]:
# 注入知识
gbi_nl2sql.knowledge["利润率"] = "计算方式: 利润/销售额"

In [11]:
nl2sql_result_message3 = gbi_nl2sql(Message({"query": "列出商品类别是日用品的利润率"}))
print(f"sql: {nl2sql_result_message3.content.sql}")
print("-----------------")
print(f"llm result: {nl2sql_result_message3.content.llm_result}")

sql: SELECT 商品类别, SUM(利润)/SUM(销售额) AS 利润率
FROM supper_market_info
WHERE 商品类别 = '日用品'
GROUP BY 商品类别
-----------------
llm result: ```sql
SELECT 商品类别, SUM(利润)/SUM(销售额) AS 利润率
FROM supper_market_info
WHERE 商品类别 = '日用品'
GROUP BY 商品类别
```


## 调整 prompt 模版
有时候，我们希望定义自己的prompt, 选表和问表两个环节都支持 prompt 模版的定制化，但是必须遵循对应的 prompt 模版的格式。

### 选表 prompt 调整
选表的 prompt template, 必须包含 
1. {num} - 表的数量， 注意 {num} 有两个地方出现
2. {table_desc} - 表的描述
3. {query} - query

注意: {num}, {table_desc}, {query} 表示的是占位符，**用户不需要在自定义的 prompt template 中将这些值填充上**，gbi 系统会自动根据 SelectTable 构造函数中提交的参数进行填充这些占位符，从而产生最后的给大模型的 prompt。注意 prompt template 和 prompt 的区别。

* prompt template - 是带有占位符的 prompt, gbi 会根据具体参数填充到这些占位符，形成最终的 prompt
* promt - 是将 prompt template 填充完占位符的结果。

用户可以使用这些占位符重新设置自己的 prompt 模版，从而达到修改 prompt 的目的。
具体请参考下面的 prompt template 示例:

In [12]:
SELECT_TABLE_PROMPT_TEMPLATE = """
你是一个专业的业务人员，下面有{num}张表，具体表名如下:
{table_desc}
请根据问题帮我选择上述1-{num}种的其中相关表并返回，可以为多表，也可以为单表,
返回多张表请用“,”隔开
返回格式请参考如下示例：
问题:有多少个审核通过的投运单？
回答: ```DWD_MAT_OPERATION```
请严格参考示例只不要返回无关内容，直接给出最终答案后面的内容，分析步骤不要输出
问题:{query}
回答:
"""

In [13]:
select_table4 = appbuilder.SelectTable(model_name="ERNIE-Bot 4.0", 
                                          table_descriptions=table_descriptions,
                                          prompt_template=SELECT_TABLE_PROMPT_TEMPLATE)

select_table_result_message4 = select_table4(Message({"query":"列出超市中的所有数据"}))
print(f"选的表是: {select_table_result_message4.content}")

选的表是: ['supper_market_info']


## 问表 prompt 调整
问表的 prompt template 必须包含:
1. {schema} - 表的 schema 信息, gbi系统使用构建 NL2Sql 对象的 table_schemas 成员来填充该占位符，在构造函数中需要填充该参数。
2. {instrument} - 列选限制的信息, gbi系统会使用 `NL2Sql.__call__` 函数的 Message 中的 column_constraint 参数来填充该占位符
3. {kg} - 知识, gbi系统使用构建 NL2Sql 对象的 knowledge  成员来填充该占位符，在构造函数中需要填充该参数。
4. {date} - 时间， gbi系统会自动填充该占位符, 用户不需要提供
5. {history_prompt} - 历史, gbi系统会使用 `NL2Sql.__call__` 函数的  Message 中的 session 参数来填充该占位符
6. {query} - 当前问题，  gbi系统会使用 `NL2Sql.__call__` 函数的 Message 中的 query 参数来填充该占位符


注意: {schema}, {instrument}, {kg}, {date}, {history_prompt}, {query} 表示的是占位符，**用户不需要在自定义的 prompt template 中将这些值填充上**，gbi 系统会自动根据 `NL2Sql.__call__` 函数中提交的参数 或者 Nl2sql 的成员变量 进行填充这些占位符，从而产生最后的给大模型的 prompt。注意 prompt template 和 prompt 的区别。

* prompt template - 是带有占位符的 prompt, gbi 会根据具体参数填充到这些占位符，形成最终的 prompt
* promt - 是将 prompt template 填充完占位符的结果。

用户可以使用这些占位符重新设置自己的 prompt 模版，从而达到修改 prompt 的目的。
具体请参考下面的 prompt template 示例:

In [14]:
NL2SQL_PROMPT_TEMPLATE = """
  MySql 表 Schema 如下:
  {schema}
  请根据用户当前问题，联系历史信息，仅编写1个sql，其中 sql 语句需要使用```sql ```这种 markdown 形式给出。
  请参考列选信息：
  {instrument}
  请参考知识:
  {kg}
  当前时间：{date}
  历史信息如下:
  {history_prompt}
  当前问题："{query}"
  回答：
"""

In [15]:

gbi_nl2sql5 = appbuilder.NL2Sql(model_name="ERNIE-Bot 4.0", table_schemas=table_schemas, prompt_template=NL2SQL_PROMPT_TEMPLATE)
nl2sql_result_message5 = gbi_nl2sql5(Message({"query": "查看商品类别是水果的所有数据"}))
print(f"sql: {nl2sql_result_message5.content.sql}")
print("-----------------")
print(f"llm result: {nl2sql_result_message5.content.llm_result}")

sql: SELECT * FROM supper_market_info WHERE 商品类别 = '水果'
-----------------
llm result: ```sql
SELECT * FROM supper_market_info WHERE 商品类别 = '水果'
```
